In [2]:
def biexpfit( xdata, ydata):

    """
    Fit sum of 2 exp with a = 0
    """
    
    import sys
    import argparse

    import numpy as np
    from math import sqrt
    from scipy.linalg import lstsq
    from scipy.optimize import curve_fit

    x = np.array(xdata)
    y = np.array(ydata)
    S = np.empty_like(y)
    S[0] = 0
    S[1:] = np.cumsum(0.5 * (y[1:] + y[:-1]) * np.diff(x))
    
    SS = np.empty_like(y)
    SS[0] = 0
    SS[1:] = np.cumsum(0.5 * (S[1:] + S[:-1]) * np.diff(x))
    
    
    x2 = x * x
    x3 = x2 * x
    x4 = x2 * x2

    M = [[sum(SS*SS),  sum(SS*S), sum(SS*x), sum(SS)],
         [sum(SS*S),   sum(S*S),  sum(S*x2), sum(S) ],
         [sum(SS*x),   sum(S*x),  sum(x2),  sum(x) ],
         [sum(SS),     sum(S),    sum(x),   len(x) ]]
    Ycol = np.array( [ sum(SS*y), sum(S*y), sum(x*y), sum(y) ] )
    (A,B,D,E),residues,rank,singulars = list( lstsq( M, Ycol ) )


    """
    Minv = np.linalg.inv(M)    
    A,B,D,E = list( np.matmul(Minv,Ycol) )
    """

    p = (1/2)*(B + sqrt(B*B+4*A))
    q = (1/2)*(B - sqrt(B*B+4*A))
    

    beta = np.exp(p*x)
    eta = np.exp(q*x)

    betaeta = beta * eta

    L = [
          [ sum(beta*beta), sum(betaeta) ],
          [ sum(betaeta), sum(eta*eta)] ]

    Ycol = np.array( [ sum(beta*y), sum(eta*y) ] )

    (b,c),residues,rank,singulars = list( lstsq( L, Ycol ) )    

    '''
    Linv = np.linalg.inv(L)
    b,c = list( np.matmul( Linv, Ycol ) )
    '''

    # sort in ascending order (fastest negative rate first)
    (b,p),(c,q) = sorted( [[b,p],[c,q]], key=lambda x: x[1])

    return b,c,p,q

In [3]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import xlsxwriter
os.chdir(r"D:\2P_fly_proboscis")
root = r'D:\2P_fly_proboscis'
datafolder = 'original data'
figfolder = 'fig_20230706'
if not os.path.exists(figfolder):
    os.makedirs(figfolder)
fitfolder = 'fig_20230706'
if not os.path.exists(fitfolder):
    os.makedirs(fitfolder)

#print(os.listdir(root + '\\' + datafolder))
#print(os.path.exists(figfolder))

In [18]:
writer = pd.ExcelWriter(os.path.join(root,'Summary.xlsx'), engine='xlsxwriter')
workbook = writer.book
datapath = os.path.join(root, datafolder)
fnames = [i for i in os.listdir(datapath) if '.csv' in i]
figpath = os.path.join(root, figfolder)
fitpath = os.path.join(root, fitfolder)
#print(fnames)
start = 500
fitnum = 1500

for i in range(0,len(fnames)):     
    #import data
    fname = fnames[i]
    print(fname)
    rawdata = pd.read_csv(os.path.join(datapath,fname))
    end = start + fitnum - 1
    df = rawdata.loc[start:end,:].copy()
    #print(type(df))
    print(df.shape[1])
    for j in range(1, df.shape[1]):
        #fit data
        b,c,p,q = biexpfit(np.array(df['Timestamp']), np.array(df.iloc[:, j]))
        #get yhat
        rawdata['yhat'+str(j)] = b * np.exp(p*rawdata['Timestamp']) + c * np.exp(q * rawdata['Timestamp'])
        #get y/yhat
        rawdata['y/yhat'+str(j)] = rawdata['Region'+' '+str(j)]/rawdata['yhat'+str(j)]    
        #get Delta F/F
        F0 = np.mean(rawdata['y/yhat'+str(j)].tolist()[1700-start:1800-start])
        rawdata['Delta F/F'+str(j)] = [(F0 - i)/F0 for i in rawdata['y/yhat'+str(j)].tolist()]
        #get smoothened Delta F/F by substrating moving average
        def moving_average(data, window_size):
        # Pad the data with NaN values to handle edges
            padded_data = np.pad(data, (window_size - 1, 0), mode='constant', constant_values=np.nan)   
            # Create the moving window view of the data
            shape = (data.size, window_size)
            strides = (data.itemsize, data.itemsize)
            windowed_data = np.lib.stride_tricks.as_strided(padded_data, shape=shape, strides=strides) 
            # Calculate the moving average
            return np.nanmean(windowed_data, axis=1)

        #define window size
        window_size = 30
        #transfer pandas series to Numpy arrays
        #print(type(rawdata['Delta F/F']))
        NpDelta = np.array(rawdata['Delta F/F'+str(j)])
        #print(type(NpDelta))
        #calculate moving average of Delta F/F
        ave = moving_average(NpDelta, window_size)

        rawdata['flattened Delta F/F'+str(j)] = rawdata['Delta F/F'+str(j)]-ave
        rawdata.to_csv(os.path.join(fitpath, fname[:-4] + '_fit.csv'), index = False)
            
        rawdata.to_excel(writer, sheet_name=fname[:-4],  index=False)
        worksheet = writer.sheets[fname[:-4]] 
        
     
        # #plot original data                                    
        # plt.figure(figsize=(8,4), dpi = 200)
        # plt.plot(rawdata['Timestamp'], rawdata['Region'+' '+str(j)],label = 'y')
        # plt.legend()
        # plt.title(label = 'Original Data'+str(j))
        # fig1= os.path.join(figpath, fname[:-4] + '_orig' +str(j)+'.png')
        # plt.savefig(fig1)
        # plt.clf()
            
        # #plot y and yhat on fitted data
                            
        # plt.figure(figsize=(8,4), dpi = 200)
        # plt.plot(rawdata['Timestamp'][start:start+fitnum], rawdata['Region'+' '+str(j)][start:start+fitnum],label = "y")
        # plt.plot(rawdata['Timestamp'][start:start+fitnum], rawdata['yhat'+str(j)][start:start+fitnum], label = "yhat")
        # plt.legend()
        # plt.title(label = ' y and yhat - fitted start from ' + str(start) + ' to ' + (str(start + fitnum)))
        # fig2 = os.path.join(figpath, fname[:-4] + '_yhat' +str(j)+'.png')
        # plt.savefig(fig2)
        # plt.clf()
                            
        # #plot Delta F/F for fitted region
        # plt.ylim([-0.5,0.5])
        # plt.plot(rawdata['Timestamp'][start:start+fitnum], rawdata['Delta F/F'+str(j)][start:start+fitnum], label = "delta F/F", color = 'black', linewidth = 0.5)
        # plt.plot(rawdata['Timestamp'][start:start+fitnum], np.zeros(fitnum), color = 'red')
        # plt.plot(rawdata['Timestamp'][start:start+fitnum], [0.1]*fitnum, color = 'red', linestyle = 'dashed')
        # plt.plot(rawdata['Timestamp'][start:start+fitnum], [-0.1]*fitnum, color = 'red', linestyle = 'dashed')
        # plt.legend()
        # plt.title(label = 'Delta F/F')
        # fig3 = os.path.join(figpath, fname[:-4] + '_deltaF' +str(j)+'.png')
        # plt.savefig(fig3)
        # plt.clf()     
                            
        # #plot all Delta F/F
        # plt.ylim([-0.5,0.5])
        # plt.plot(rawdata['Timestamp'], rawdata['Delta F/F'+str(j)], label = "delta F/F", color = 'black')
        # plt.title(label = 'all Delta F/F'+str(j))
        # fig4 = os.path.join(figpath, fname[:-4] + '_deltaF' + 'all' +str(j)+'.png')
        # plt.savefig(fig4)
        # plt.clf()

        #plot flattened Delta F/F
        plt.ylim([-0.5,0.5])
        plt.plot(rawdata['Timestamp'][start:], rawdata['flattened Delta F/F'+str(j)][start:], label = "flattened Delta F/F", color = 'black', linewidth = 0.5)
        plt.plot(rawdata['Timestamp'][start:], [0.1]*(rawdata['Timestamp'].shape[0]-start), color = 'red', linestyle = 'dashed')
        plt.plot(rawdata['Timestamp'][start:], [-0.1]*(rawdata['Timestamp'].shape[0]-start), color = 'red', linestyle = 'dashed')
        plt.title(label = 'flattened Delta F/F'+str(j))
        fig5 = os.path.join(figpath, fname[:-4] + '_flattened deltaF' + 'all' +str(j) +'.png')
        plt.savefig(fig5)
        plt.clf()

        plt.close('all')   
        
#writer.save()
workbook.close()

control PD male 3-038.csv
3
control_PD_male_1_030.csv
8


C:\Users\bdljq\AppData\Local\Temp\ipykernel_15132\927120415.py:39: RuntimeWarning: Mean of empty slice
  return np.nanmean(windowed_data, axis=1)


control_PD_male_4-039.csv
3
control_PD_male_6-050.csv
3
control_SAT_female-033.csv
8
control_SAT_male_2-045.csv
4
mutant_SAT_male_1-029.csv
8
mutant_SAT_male_2-088.csv
3
mutant_SAT_male_4-091.csv
4
